In [1]:
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
root_dir = os.path.abspath('./')
# root_dir = os.path.abspath('./drive/My Drive/C1/Project_C1/')
# Set the directory of the training data
train_dir = os.path.join(root_dir, 'Training')

In [3]:
train_files = pd.read_csv(os.path.join(root_dir, 'train_files.csv'))

In [4]:
# set constants
COLS, ROWS, CHANNEL = 480, 640, 3

In [5]:
temp = []
n=0
for img_name in train_files.file_name:
    image_path = os.path.join(train_dir, img_name)
    img = cv2.imread(image_path)
    # img = np.expand_dims(cv2.resize(img, dsize=(COLS, ROWS), interpolation=cv2.INTER_CUBIC), axis=2)
    # .flatten()
    # print(n)
    # print(type(img))
    # print(img.shape)
    # cv2.imshow('image', img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    img = img.astype('float32')
    temp.append(img)
    n+=1

In [6]:
X_train = np.stack(temp)

In [7]:
y_train = train_files.annotation.values

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1, shuffle=True, stratify=y_train)

In [9]:
# models = [tf.keras.applications.Xception(include_top=True,
#                                          weights=None,
#                                          input_tensor=None,
#                                          input_shape=(COLS, ROWS, CHANNEL),
#                                          pooling=None,
#                                          classes=5),
          
#          tf.keras.applications.InceptionV3(include_top=True,
#                                             weights=None,
#                                             input_tensor=None,
#                                             input_shape=(COLS, ROWS, CHANNEL),
#                                             pooling=None,
#                                             classes=5) ]


def custom_model(layer1_size, layer2_size, layer3_size, dropout=0.4):
    
    # initialize the model from the keras library
#     custom_model = tf.keras.applications.VGG16(include_top=False,
#                                               weights='imagenet',
#                                               input_tensor=None,
#                                               input_shape=(COLS, ROWS, CHANNEL),
#                                               pooling=None,
#                                               classes=5)
    
    custom_model = tf.keras.applications.Xception(include_top=False,
                                         weights='imagenet',
                                         input_shape=(480, 640, 3),
                                         classes=5)
    
#     # Top Model Block
#     x = custom_model.output
#     x = tf.keras.layers.GlobalAveragePooling2D()(x)
#     predictions = tf.keras.layers.Dense(5, activation='softmax')(x)

#     # add your top layer block to your base model
#     model = tf.keras.Model(custom_model.input, predictions)
    
    
    model = tf.keras.Sequential()

    # Freeze the layers
    for layer in custom_model.layers:
        layer.trainable = False
    
    model.add(custom_model)
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(layer1_size, activation='relu'))
    model.add(tf.keras.layers.Dropout(dropout))
    model.add(tf.keras.layers.Dense(layer2_size, activation='relu'))
    model.add(tf.keras.layers.Dropout(dropout))
    model.add(tf.keras.layers.Dense(layer3_size, activation='relu'))
    model.add(tf.keras.layers.Dropout(dropout))
    # Add 'softmax' instead of earlier 'prediction' layer.
    model.add(tf.keras.layers.Dense(5, activation='softmax'))

    return model

In [10]:
# Hyperparams

layer1 = [512, 256]
layer2 = [512, 256]
layer3 = [256]

dropout = [0.3, 0.4, 0.5]
activation = ["relu"]

EPOCHS = 30
BATCH_SIZE = 16

# plot directory for accuracy and loss
validation = "./transfer_learning/validation/results{}_{}_{}_{}_{}.txt.".format(layer1, layer2, layer3, dropout, activation)
plot_acc = "./transfer_learning/acc/"
plot_loss = "./transfer_learning/loss/"

os.makedirs(plot_acc, exist_ok=True)
os.makedirs(plot_loss, exist_ok=True)

In [11]:
f = open(validation, "w+")

n=0

for l1 in layer1:
    for l2 in layer2:
        for l3 in layer3:
            for dr in dropout:
                print("-------------------")
                print("Compiling model with {}, {}, {}, {}".format(l1, l2, l3, dr))
                print("-------------------")

                # initialize the model
                model = custom_model(l1, l2, l3, dr)

                model.summary()
                model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
                                       optimizer=tf.keras.optimizers.Adam(),
                                       metrics=['accuracy'])


                print("-------------------")
                print("Training model with {}, {}, {}, {}".format(l1, l2, l3, dr))
                print("-------------------")


#                 with tf.device('/device:GPU:0'):

                history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(X_val, y_val))

                acc = history.history['acc']
                val_acc = history.history['val_acc']
                loss = history.history['loss']
                val_loss = history.history['val_loss']


                plt.figure(n)
                plt.suptitle('Accuracy learning curve', fontsize=20)
                plt.xlabel('epochs', fontsize=14)
                plt.ylabel('accuracy', fontsize=14)
                plt.plot(acc, label='training accuracy')
                plt.plot(val_acc, label='validation accuracy')
                plt.xticks(np.arange(0, EPOCHS, EPOCHS/10))
                plt.legend(loc="lower right")
                plt.savefig("{}_{}_{}_{}_{}.png".format(plot_acc, l1, l2, l3, dr), dpi=500)

                plt.figure(n+500)
                plt.suptitle('Loss', fontsize=20)
                plt.xlabel('epochs', fontsize=14)
                plt.ylabel('loss', fontsize=14)
                plt.plot(loss, label='training loss')
                plt.plot(val_loss, label='validation loss')
                plt.xticks(np.arange(0, EPOCHS, EPOCHS/10))
                plt.legend(loc="upper right")
                plt.savefig("{}_{}_{}_{}_{}.png".format(plot_loss, l1, l2, l3, dr), dpi=500)


                f.write("layer1: {}, layer2: {}, layer4: {}, Dropout: {}\n".format(l1, l2, l3, dr))
                f.write("Validation accuracy = {}\n".format(val_acc))
                f.write("----------------------\n")
                f.write("Validation loss = {}\n".format(val_loss))
                f.write("----------------------\n")
                f.write("----------------------\n\n")

                n+=1

            
f.close()                    

-------------------
Compiling model with 512, 512, 256, 0.3
-------------------
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 15, 20, 2048)      20861480  
_________________________________________________________________
flatten (Flatten)            (None, 614400)            0         
_________________________________________________________________
dense (Dense)                (None, 512)               314573312 
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
__________________________________________

KeyboardInterrupt: 